# Problem 6

## GNN for node classification and link prediction

In [1]:
# Import the libraries

import numpy as np

from tqdm import tqdm
from matplotlib import pyplot as plt

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.utils.data as Data
import matplotlib.pyplot as plt

from torch_geometric.datasets import Planetoid
import networkx as nx 

SEED = 0
torch.manual_seed(SEED)
np.random.seed(SEED)

torch.set_default_dtype(torch.float64)

In [2]:
dataset = Planetoid(root='/data', name='PubMed')

In [3]:
data = dataset[0]

# Let's have a look to how data are structured

data

Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])

### Exploratory data analysis

First of all, let's build the graph underlying our dataset to retrieve some precious informations. 

In [14]:
nodes_TFIDF = data.x
nodes_classes = data.y

nodes = torch.arange(0, nodes_TFIDF.shape[0])
edges = data.edge_index

Let's remap the tensor containing the edges to a data structure digestible by ```NetworkX```, that requires the edges to be 2-tuples:

In [21]:
edges

tensor([[ 1378,  1544,  6092,  ..., 12278,  4284, 16030],
        [    0,     0,     0,  ..., 19714, 19715, 19716]])

In [22]:
edges_tail = edges[0,:]
edges_head = edges[1,:]

edges = [(edges_tail[i], edges_head[i]) for i in range(edges.shape[1])]

In [23]:
PubMed_graph = nx.DiGraph()

PubMed_graph.add_nodes_from(nodes.detach().numpy())
PubMed_graph.add_edges_from(edges)